<a href="https://colab.research.google.com/github/MDKate/CBR/blob/main/%D0%A7%D0%91%D0%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import telebot
import telegram
from telebot import types
from telebot.types import ReplyKeyboardMarkup, InlineKeyboardMarkup, InlineKeyboardButton
import pandas as pd
import numpy as np
import emoji

In [ ]:
bot = telebot.TeleBot('5958215181:AAFSaPDPJr9JFxtT3UWkO_WWFxTQMEQ2DE8')

In [ ]:
@bot.message_handler(commands=['start'])  # Начинаем работу

def start(m):

    global count
    global page
    markup = InlineKeyboardMarkup()  # Определяем кнопку
    markup.add(InlineKeyboardButton(text=f'Начнем', callback_data=f'start'))
    bot.send_message(m.from_user.id, emoji.emojize(
        "Добрый день!:hand_with_fingers_splayed: Давайте начнем учиться! :play_button:"),
                    reply_markup=markup)  # Выводим сопутствующее сообщение


@bot.message_handler()  # Обрабатываем текстовые сообщения

def start(m):
    bot.send_message(m.from_user.id, emoji.emojize(
        "Увы! :weary_face: Извините! Я еще плохо умею общаться 	:woman_facepalming:"))  # Выводим сопутствующее сообщение

if __name__ == '__main__':

    bot.polling(none_stop=True)
